Notebook version 1.0, 31 Aug 2021. Written by Otto Salmenkivi / CSC - IT Center for Science Ltd. otto.salmenkivi@gmail.com

Licensed under the MIT license: http://www.opensource.org/licenses/mit-license.php

Tested on Kvasi, running QLM version 1.2.1: https://research.csc.fi/-/kvasi
***
# Quantum algorithm for T-Bill expected value
Based on Woener and Egger, 2018: https://arxiv.org/abs/1806.06893 

In this notebook we implement the calculation of the expectation value of a US Treasury bill. The zero coupon T-bill has a face value $V_F$ and is discounted at an interest rate $r$.  We want to calculate the value at a later time step given that there is a $p$ probability that the interest is constant, and $1-p$ probability that the interest rises to $r+\delta r$. Therefore the T-bill has a value of
<div class='math'>
            \begin{equation}
            V = \dfrac{(1-p)V_F}{1+r+\delta r} + \dfrac{pV_F}{1+r} = (1-p)V_{\text{low}} + pV_\text{high}.
            \end{equation}

We set $p = 30\%$. The values for $V$ need to be mapped to qubits. In this simplified demonstration in order to use only one qubit for the distribution of $V$, we map $ V_\text{low} = 0\: \$ $ and $ V_\text{high} = 1\: \$ $. These correspond to the states $|0\rangle$ and $|1\rangle$, respectively.
    
We use Quantum Amplitude Estimation to calculate the value of the T-bill. QAE is build from four components:
- initial superposition of evaluation qubits with Hadamard gates
- an $A$ gate that encodes the distribution of the possible outcomes for $V$ with corresponding probabilities and the objective function
- a $Q$ gate based on $A$ and the number of evaluation qubits
-  an inverse Quantum Fourier Transform.
    
In our simple case the $A$ gate is defined as $A =R_y(\theta)$, where $\theta = 2\sin^{-1}(\sqrt{p})$, i.e. it's merely a rotation around the y-axis parametrized by the probability $p$. Therefore it acts as $A|0\rangle = \sqrt{1-p}|0\rangle + \sqrt{p}|1\rangle$, and respresents both the uncertainty distribution and the objective of our algorithm. Another gate needed is the amplitude estimation operator $Q$. Again, for us it simplifies to $Q = AZA^{\dagger}Z = R_y(2\theta)$. Powers of this gate are controllably applied $m$ times to the objective qubit, where $m$ is the number of evaluation qubits. These qubits are responsible for the accuracy and resolution of the estimate. The controlled gates are of form $ Q^{2^j} = R_y(2^{j+1}\theta)$, where $j \in \{0,\ldots,m-1\}$.
   

We begin by importing the libraries needed.

In [ ]:
from qat.lang.AQASM import Program, QRoutine, H, RY, SWAP
from qat.lang.AQASM.qftarith import QFT
import numpy as np

In the following we define two functions that return `QRoutine` objects. First applies a Hadamard gate to each evaluation qubit and the second builds the controlled $Q$ gates.

In [ ]:
def hadamard_all(nbqbits):
    rout = QRoutine()
    for i in range(nbqbits):
        rout.apply(H,i)
    return rout

In [ ]:
# Function for creating the Q^2^j gates
def Q_gates(m):
    rout = QRoutine()
    wires = rout.new_wires(m)
    single_wire = rout.new_wires(1)
    for i in range(m):
        angle = 2**(i+1)*theta_p % (4*np.pi)  # the modulo operator % is not necessary. RY repeats every 4*pi
        rout.apply(RY(angle).ctrl(), wires[i], single_wire)
    return rout

The probalitity $p$ and number of evaluation qubits $m$ can be changed below. The circuit is build based on those values. 

In [ ]:
p = 0.3                            # Probability
theta_p = 2*np.arcsin(np.sqrt(p))  # And the related angle for Q gates
m = 7                              # Number of evaluation qubits

# initiation
prog = Program()
m_qbits = prog.qalloc(m)
single_qbit = prog.qalloc(1)

# Hadamard to all evaluation qubits
prog.apply(hadamard_all(m),m_qbits)

# The A gate 
prog.apply(RY(theta_p),single_qbit)

# Applying Q gates
prog.apply(Q_gates(m), m_qbits , single_qbit)

# Inverse Quantum Fourier Transform
prog.apply(QFT(m).dag(),m_qbits)

circuit = prog.to_circ()
%qatdisplay circuit

The circuit for the algorithm is now ready. Let's run it on the simulator and print the initial results. The useful information is stored in the evaluation qubits, so we ignore the last qubit in the measurement.

In [ ]:
# Running the circuit
from qat.qpus import LinAlg
nbshots = 0
job = circuit.to_job(qubits = m_qbits, nbshots = nbshots)
result = LinAlg().submit(job)

for sample in result:
    print(f'State: {sample.state},  amplitude: {sample.amplitude},  probability: {sample.probability}')

# Visualizing the initial measurement results
import matplotlib.pyplot as plt
plt.figure()
states = [str(sample.state) for sample in result]
probs = [sample.probability for sample in result]
plt.bar(states,probs)
plt.ylabel('Probability')
plt.xticks(rotation = 60)
plt.draw()

The measurement results correspond to integers $y \in \{0,\ldots,M-1\}$, where $M = m^2$. These a mapped to the estimator $\tilde{p} = \sin^2(y\pi/M) \in [0,1]$. The estimator converges to $p$ in $O(M^{-1})$, which is quadratically faster than classical Monte Carlo methods and at the root of the proposed computational speed-up.

We do the post-processing below. Since the results contain only measured states, we create a new array and fill in the missing zero amplitudes. It is also noteworthy that multiple states correspond to the same estimator value due to the periodic nature of the $sin$ function.


In [ ]:
# Creating an array of zeros for all state probabilities and replacing the non-zero values for measured probabilities 
probs = np.zeros(2**m, dtype=float)
for sample in result:
    state_decimal = sample.state.int
    probs[state_decimal] = sample.probability

# The mapping used between measured states and corresponding estimate for the probability p
p_tilde = [np.round(np.sin(i*np.pi/(2**m))**2,3) for i in range(2**(m-1)+1)]
print(f'Possible values for the approximation: \n {p_tilde}')
print(f'{len(p_tilde)} possibilities in total.')
# Aggregating the data from different states that correspond to sama probability bins
freqs =[]
freqs.append(probs[0])
i = 1
while i < 2**m/2:
    # print(f'These states correspond to the same bin: {i} and {2**m-i}')
    freqs.append(probs[i] + probs[2**m-i])
    i += 1
freqs.append(probs[2**(m-1)])

# finding p with max frequency
max_freq = p_tilde[np.argmax(freqs)]

That's all the computation and calculation we need to do. Let's finish the notebook by printing the circuit again and by visualizing the final results.

In [ ]:
print(f'Probability was set to {p} and {m} evaluation qubits were used.')
print('The circuit built for the estimation on T-bill value using QAA:')
%qatdisplay circuit

print(f'The highest peak is at p = {max_freq}.')

print('The final results after calculating the mapping.\n The red line represents the target value.')
plt.figure()
plt.bar(p_tilde, freqs, width = 0.01, color = 'royalblue')
plt.ylim(0,1)
#plt.xlim(0.1,0.5)
plt.axvline(x=0.3, color = 'r', linestyle = '--', linewidth = 3 )
plt.xlabel(r'$\tilde{p}$',fontsize = '12')
plt.ylabel(' Todennäköisyys ',fontsize = '12')
plt.tick_params(axis='both', which='major', labelsize='11')
textstr = '\n'.join((f'$m=${m}',f'$M=${2**m}', r'$\tilde{p} \approx %.3f$' % (max_freq, )))
plt.text(0.4,0.75, textstr, fontsize = '13')
filename = f'velkakujaava_m{m}'
#plt.savefig(filename, dpi = 300)
plt.plot()

A clear peak close to the set probability $p$ should be visible. Resolution and accuracy grows with evaluation qubits. Since we had originally mapped our two possible states $|0\rangle$ and $|1\rangle$ to values $0 \$ $ and $1 \$$ respectively, we can trivially calculate the expected value. For $p = 0.3$ and $m = 5$, QAA produces a approximate value of $\tilde{p} = 0.309$. Therefore the expectation value
    \begin{equation}
        V = (1 - 3.09)\cdot 0 \$ + 0.309 \cdot 1 \$ = 0.309 \$ \,.
    \end{equation}

This was a simple demonstration of the Quantum Amplitude Estimation algorithm applied to calculating the expactation value of a US Treasury bill, meant to show the key parts of the method. Next step could be to implement a more comprehensive distribution of possible outcomes, which would mean a non-trivial objective operator acting on an additional qubit as well as more complex $A$ and $Q$ gates. That's a story for another time.